In [11]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Cargar Datos

In [6]:

with open('entrenamiento.p', 'rb') as file:
    training_data = pickle.load(file)
    

with open('prueba.p', 'rb') as file:
    test_data = pickle.load(file)
    

with open('validacion.p', 'rb') as file:
    validation_data = pickle.load(file)


# Dividir la data en X, Y

In [26]:
X_train = training_data["features"]
y_train = training_data["labels"]

In [27]:
X_test = test_data["features"]
y_test = test_data["labels"]

In [28]:
X_val = validation_data["features"]
y_val = test_data["labels"]

In [29]:
X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

array([[41],
       [41],
       [41],
       ...,
       [25],
       [25],
       [25]], dtype=uint8)

In [38]:
y_train.shape

(34799,)

In [36]:
from tensorflow.keras.utils import to_categorical

In [37]:
y_cat_train = to_categorical(y_train, 43)

IndexError: index 42 is out of bounds for axis 1 with size 42